## Работа со встроенным датасетом fetch_20newsgroups

In [2]:
from sklearn.datasets import fetch_20newsgroups

In [43]:
news=fetch_20newsgroups()

## Изучаю данные

Датасет представляет собой словарь (совокупность нескольких "пакетов" данных)

In [46]:
news.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR', 'description'])

В спеке на scikit описано, что датасет разделен на тренировочное и тестовое подмножество. Как я выяснила, по умолчанию подгружается тренировочное подмножество

In [44]:
len(news.data)

11314

In [42]:
check2=fetch_20newsgroups(subset='all')
len(check2.data)

18846

In [45]:
check1=fetch_20newsgroups(subset='train')
len(check1.data)

11314

In [18]:
type(news.filenames)

numpy.ndarray

Датасет <b>news</b> по умолчанию содержит подмножества <b>DESCR</b> (NoneType), <b>data</b> (list), <b>description</b> (str), <b>description</b> (numpy.ndarray), <b>targat</b> (numpy.ndarray), <b>target_names</b> (list)

В датасете есть следующие категории писем:

In [12]:
news.target_names  

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

проверила, что их 20

In [29]:
import numpy as np
np.unique(news.target)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

Письма содержатся в подмножестве data. Далее я для примера вывела одно письмо. Изначально они представлены в неудобном для чтения виде, поэтому я разделила письмо по символу \n с помощью функции split

In [20]:
for line in news.data[15].split('\n'):
        print(line)
    

From: mathew <mathew@mantis.co.uk>
Subject: Re: <Political Atheists?
Organization: Mantis Consultants, Cambridge. UK.
X-Newsreader: rusnews v1.01
Lines: 22

kmr4@po.CWRU.edu (Keith M. Ryan) writes:
> ( I am almost sure that Zyklon-B is immediate and painless method of 
> death. If not, insert soem other form. )
> 
>         And, ethnic and minority groups have been killed, mutilated and 
> exterminated through out history, so I guess it was not unusual.
> 
>         So, you would agree that the holocost would be allowed under the US 
> Constitution?  [ in so far, the punishment. I doubt they recieved what would 
> be considered a "fair" trial by US standards.

Don't be so sure.  Look what happened to Japanese citizens in the US during
World War II.  If you're prepared to say "Let's round these people up and
stick them in a concentration camp without trial", it's only a short step to
gassing them without trial.  After all, it seems that the Nazis originally
only intended to imprison the

Всего в подмножестве news.data содержится 11314 писем

In [26]:
len(news.data)

11314

Для удобства работы, я создала датафрейм на основе датасета. (это вовсе не обязательно, просто я решила так сделать)

In [30]:
import pandas as pd

ddata=pd.DataFrame(data=news.data,columns=['Message'])
ddata['target']=news.target

In [31]:
ddata.head(10)

,Message,target
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14
5,From: dfo@vttoulu.tko.vtt.fi (Foxvog Douglas)\...,16
6,From: bmdelane@quads.uchicago.edu (brian manni...,13
7,From: bgrubb@dante.nmsu.edu (GRUBB)\nSubject: ...,3
8,From: holmes7000@iscsvax.uni.edu\nSubject: WIn...,2
9,From: kerr@ux1.cso.uiuc.edu (Stan Kerr)\nSubje...,4


По заданию, мне нужно было выбрать 4 категории и классифицировать письма согласно этим категориям.
Я выбрала Автомобили, Медицину, Космос и Религию (Христианство)
Когда я это сделала, в первой колонке остались старые индексы. Поэтому я применила функцию reset_index, чтобы индексы установить заново
Получилась таблица, которая соотносит оставшиеся 2380 писем с их таргетами (номерами категорий)

In [49]:
catdata=ddata.loc[ddata['target'].isin([7, 13, 14, 15])]
catdata=catdata.reset_index(drop=True)
print(catdata.shape)
catdata[:10]

(2380, 2)


,Message,target
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7
1,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14
2,From: bmdelane@quads.uchicago.edu (brian manni...,13
3,From: dbm0000@tm0006.lerc.nasa.gov (David B. M...,14
4,From: CPKJP@vm.cc.latech.edu (Kevin Parker)\nS...,7
5,From: jonh@david.wheaton.edu (Jonathan Hayward...,15
6,From: jimf@centerline.com (Jim Frost)\nSubject...,7
7,Subject: Teenage acne\nFrom: pchurch@swell.act...,13
8,From: 18084TM@msu.edu (Tom)\nSubject: Golden &...,14
9,From: dlecoint@garnet.acns.fsu.edu (Darius_Lec...,15


### Далее я изучала основные понятия, которые используют при анализе текстовых данных.

#### n-граммы
Прежде всего я создавала n-граммы, которые явялются одним из основных понятий в анализе текстовых данных. 
n-граммы, по сути, являются последовательностями сабстрингов длины n исходного текста. 
Например, если у нас есть текст, состоящий из 5 слов (от 1 до 5), то n-граммой "по словам" при n=3 , будет являтся последовательность слов (1,2,3), (2,3,4), (3,4,5). Также есть n-граммы по символам. Алгоритм для них такой же, за исключением того, что в данном случае можно выделить 2 вида  n-грамм: те, которые работают только с начала и до конца слова, и те, которые игнорируют пробелы, и работают на всей последовательности символов текста.

Для создания n-грамм я взяла CountVectorizer. Он принимает на вход массив текстовых данных и возвращает вектор признаков (набор чисел). То есть я  представила признаки в числовом формате, так как классификатор работает с числовыми данными. 

Далеее я для примера преобразовала 16-е по счету письмо в тренировочной выборке в набор чисел. 
По сути, чем больше число в получившейся матрице, тем большие вес, id est большую уникальность имеет данное слово.

In [73]:
from sklearn.feature_extraction.text import CountVectorizer
vect=CountVectorizer( ngram_range=(1,1),analyzer='word')
matr=vect.fit_transform([news.data[15]])
matr.todense()

matrix([[1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 3, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 2, 3, 2, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 2,
         3, 9, 2, 1, 2, 1, 1, 5, 1, 3, 2, 1, 1, 2, 3, 1, 2, 2, 1, 2, 1, 2,
         1, 3, 1, 2, 1]], dtype=int64)

#### tf-idf

Далее, я использовала tf-idf.

tf-idf применяется, когда у нас имеется набор (коллекция) текстов, в данном случае - писем. tf-idf оценивает важность конкретного слово во всем наборе писем. Вес слова пропорционален количеству употреблений слова в письме, и обратно пропорционален количеству употреблений данного слова в остальных писмах нашего набора.

Рассчитывается: term frequency * inverted document frequency

Например:

Возьмем текст А, посчитаем количество вхождений слова 'nadz' в этот текст.
Пусть слов в тексте у нас 1000, а the  встречается 10 раз.

tf('nadz')=0.01

Чтобы посчитать inverted document frequency для этого слова, нужно взять логарифм от отношения количества текстов, в которых есть хоть одно вхождение этого слова, к общему количеству текстов. Думаю, логарифм берут для нормализации данных, чтобы не было большого разброса.

Выбор основания логарифма в формуле не имеет значения, поскольку изменение основания приводит к изменению веса каждого слова на постоянный множитель, что не влияет на соотношение весов.

Если всего текстов 10000, и слово “nadz” встретилось в 100 текстах, то

idf(“nadz”) = log(10000/100) = 4.6 

Таким образом, частые слова имеют минимальный вес, и специфичные редкие — большой

в Python для вычисления tf-idf используется TfidfTransformer.

In [94]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
X=catdata['Message']
y=catdata['target']


### Построение модели

Далее, создаем пайплайн, который по сути определет последовательность действий, которые будут выполнены.
В нашем случае, сначала мы сначала применяем к набору писем CountVectorizer, который преобразует каждое письмо в матрицу чисел, в соответствии с весом (уникальностью) каждого слова в конкретном письме. При этом, мы установили параметр stop_words - частовстречающихся слов, которые будут игнорироваться при анализе. Мы используем встроенный стоп-лист слов для англоязычных текстов 'english'.

При подборе параметров, я использовала разные функции для анализа - word, char, char_wb, word, которые отвечают за то, из чего будет стоиться n-грамма - из слов, или из символов (с учетом разделения слов пробелами, или без).

Наилучший результат получился, при построении n-грамм по словам. Это и логично, ведь тексты по тематике можно отличить прежде всего ключевыми словами.

Итак, на данном этапе мы получили матрицу, состоящую из чисел.

Далее, нам нужно нормализовать получившуюся матрицу. Для этого используем TfidfTransformer.

Норму я выбрала L2. Эта норма соответствует Евклидовой норме, которая вычисляется в случае матрицы как квадратный корень суммы квадратов каждого элемента.

На данном этапе я получила нормализованную матрицу.

Далее, я построила классификатор стохастического градиентного спуска (SGD).
penalty отвечает за функцию регуляризации, я выбрала L2, которая является Ridge регуляризацией.
В качестве loss функции используем Huber loss, которая менее чувствительная к выбросам (штрафует их поменьше). Т.е, сначала она работает как  squared error loss, а потом, начиная с определенного большого значения работает линейно. Это позволяет уменьшить "разброс"

А для подбора наиболее подходящих параметров я использовала RandomizedSearchCV. GreadSearch не взяла, потому что он использует максимальный score (может сильно отличаться от минимального, и в результате среднее значение будет хуже). А RandomizedSearch выбирает среднее значение. 
Значений перебрала много, самые оптимальные использовала дальше.

In [95]:
pipe = Pipeline([
    ('vect', CountVectorizer(stop_words = 'english',analyzer='word')),
    ('tfidf', TfidfTransformer(norm = 'l2')),
    ('clf', SGDClassifier(penalty='l2',loss='modified_huber')),
    ])
param_grid={
    'vect__ngram_range': ((1, 6),(1, 7), (3, 7),(3,6)), 
    'clf__alpha': (0.0000001,0.000001,0.00001),
    'vect__max_df':(0.3, 0.5)
}
search=RandomizedSearchCV(pipe, param_distributions=param_grid,n_iter=20)
search.fit(X, y)
score = cross_val_score(pipe, X, y, scoring='accuracy', cv=17, n_jobs=-1)
print("{} +/- {}".format(score.mean(), score.std())) 


0.9802875559338159 +/- 0.009582259367351628


функцию report использую для представления результатов. скопировала ее из спеки RandomizedSearch. Она позволяет отсортировать топ-n результатов, а также, соответственно, я вижу параметры которые "сработали" лучше, это я использовала при дальнейшем изменении параметров.

In [99]:
def report(results, n_top=5):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [103]:
report(search.cv_results_)

Model with rank: 1
Mean validation score: 0.963 (std: 0.006)
Parameters: {'vect__ngram_range': (1, 6), 'vect__max_df': 0.5, 'clf__alpha': 1e-05}

Model with rank: 2
Mean validation score: 0.959 (std: 0.008)
Parameters: {'vect__ngram_range': (1, 7), 'vect__max_df': 0.3, 'clf__alpha': 1e-05}

Model with rank: 3
Mean validation score: 0.959 (std: 0.004)
Parameters: {'vect__ngram_range': (1, 7), 'vect__max_df': 0.5, 'clf__alpha': 1e-05}

Model with rank: 4
Mean validation score: 0.958 (std: 0.007)
Parameters: {'vect__ngram_range': (1, 6), 'vect__max_df': 0.3, 'clf__alpha': 1e-06}

Model with rank: 4
Mean validation score: 0.958 (std: 0.005)
Parameters: {'vect__ngram_range': (1, 6), 'vect__max_df': 0.3, 'clf__alpha': 1e-07}



Ниже мне захотелось проверить работу моей модели на двух рандомных текстиках, найденных в гугле. Модель сработала верно.

In [104]:
a='To learn more about this FRB, scientists used the Arecibo Observatory in Puerto Rico and the Green Bank Telescope in West Virginia to analyze data on 16 bursts from object. FRB 121102 is located in a star-forming region of a dwarf galaxy found about 3 billion light-years from Earth, Hessels said. Because astronomers can see it from such a great distance, the amount of energy in a single millisecond of each of these bursts must be about as much as the sun releases in an entire day, Hessels and his colleagues said in a statement.'

print(news.target_names[search.predict([a])[0]])

b='In this phase 3, double-blind trial, which was conducted at 133 centers worldwide, we randomly assigned 1402 patients with type 1 diabetes who were receiving treatment with any insulin therapy (pump or injections) to receive sotagliflozin (400 mg per day) or placebo for 24 weeks. The primary end point was a glycated hemoglobin level lower than 7.0% at week 24, with no episodes of severe hypoglycemia or diabetic ketoacidosis after randomization. Secondary end points included the change from baseline in glycated hemoglobin level, weight, systolic blood pressure, and mean daily bolus dose of insulin'

print(news.target_names[search.predict([b])[0]])

sci.space
sci.med


### Тест

Тут я протестировала модель на тестовом подмножестве. В результате точность предсказания составила 94 %

In [105]:
news2=fetch_20newsgroups(subset='test')

In [106]:
news2.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR', 'description'])

In [107]:
ddatatest=pd.DataFrame(data=news2.data,columns=['Message'])
ddatatest['target']=news2.target

In [108]:
testdata=ddatatest.loc[ddatatest['target'].isin([7, 13, 14, 15])]
testdata=testdata.reset_index(drop=True)
print(testdata.shape)
testdata[:10]

(1584, 2)


,Message,target
0,From: v064mb9k@ubvmsd.cc.buffalo.edu (NEIL B. ...,7
1,From: banschbach@vms.ocom.okstate.edu\nSubject...,13
2,From: PETCH@gvg47.gvg.tek.com (Chuck)\nSubject...,15
3,From: fortmann@superbowl.und.ac.za (Paul Fortm...,15
4,From: prb@access.digex.net (Pat)\nSubject: Re:...,14
5,From: george@crayola.East.Sun.COM (George A. P...,13
6,From: cadiz@rtsg.mot.com (Jay Cadiz)\nSubject:...,7
7,From: rmakarem@usc.edu (Total Stranger)\nSubje...,7
8,From: baalke@kelvin.jpl.nasa.gov (Ron Baalke)\...,14
9,From: abdkw@stdvax.gsfc.nasa.gov (David Ward)\...,14


In [114]:
testpred=p.predict(testdata.Message)
testpred[:10]

array([ 7, 13, 15, 15, 14, 13,  7,  7, 14, 14])

In [110]:
from sklearn.metrics import accuracy_score
accuracy_score(testdata['target'], testpred)

0.94696969696969702